In [ ]:
import os
import pandas as pd
import torch
from transformers import RobertaModel, RobertaTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, ClassLabel
import pickle
import wandb
from lion_pytorch import Lion
from transformers import get_scheduler
from torch.optim import AdamW, Adam
from datasets import logging as datasets_logging
from transformers import logging as transformers_logging

datasets_logging.set_verbosity_error()
base_model = 'roberta-base'
dataset = load_dataset('ag_news', split='train', cache_dir = './data')
class_names = dataset.features["label"].names
id2label = {i: label for i, label in enumerate(class_names)}
transformers_logging.set_verbosity_error()
model = RobertaForSequenceClassification.from_pretrained(
    base_model,
    id2label=id2label)

# FAST TUNING ===========================================================
layer_configs = [
            {"layer": 0, "r": 2, "lora_alpha": 4, "lora_dropout": 0.01, "target": ['query']},
            {"layer": 1, "r": 2, "lora_alpha": 4, "lora_dropout": 0.01, "target": ['query']},
            {"layer": 2, "r": 4, "lora_alpha": 4, "lora_dropout": 0.03, "target": ['query']},
            {"layer": 3, "r": 4, "lora_alpha": 4, "lora_dropout": 0.03, "target": ['query']},
            {"layer": 4, "r": 8, "lora_alpha": 4, "lora_dropout": 0.05, "target": ['query']},
            {"layer": 5, "r": 8, "lora_alpha": 4, "lora_dropout": 0.05, "target": ['query']},
            {"layer": 6, "r": 8, "lora_alpha": 4, "lora_dropout": 0.05, "target": ['query']},
            {"layer": 7, "r": 8, "lora_alpha": 4, "lora_dropout": 0.05, "target": ['query']},
            {"layer": 8, "r": 12, "lora_alpha": 4, "lora_dropout": 0.075, "target": ['query']},
            {"layer": 9, "r": 12, "lora_alpha": 4, "lora_dropout": 0.075, "target": ['query']},
            {"layer": 10, "r": 12, "lora_alpha": 4, "lora_dropout": 0.075, "target": ['query']},
            {"layer": 11, "r": 12, "lora_alpha": 4, "lora_dropout": 0.075, "target": ['query']}]

for config in layer_configs:
    model = RobertaForSequenceClassification.from_pretrained(
        base_model,
        id2label=id2label
    )

    target_layer = f"encoder.layer.{config['layer']}.attention.self.query"
    peft_config = LoraConfig(
        r=config["r"],
        lora_alpha=config["lora_alpha"],
        lora_dropout=config["lora_dropout"],
        bias="none",
        target_modules=['query'],
        task_type="SEQ_CLS",
    )
    print(f"Applying LoRA to {target_layer}")
    model = get_peft_model(model, peft_config) 

peft_model = model
print(peft_model)
peft_model.print_trainable_parameters()
